<a href="https://colab.research.google.com/github/UgoRicher/ProjetTechnique/blob/main/Interface_graphique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install Flask pyngrok



In [10]:
!ngrok config add-authtoken 2hpBCmTTLROiBR517ajltO3mq4y_4LZie97eLGr3RUQCyCHiv


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
from flask import Flask, request, render_template_string, jsonify
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import io
import base64
from pyngrok import ngrok
import time
import threading

app = Flask(__name__)

# Chemin du fichier de données
file_path = '/content/Online Sales Data.csv'

# Vérifiez si le fichier existe
import os
if not os.path.exists(file_path):
    file_path = '/mnt/data/Online Sales Data.csv'
    if not os.path.exists(file_path):
        raise FileNotFoundError("Le fichier de données est introuvable dans les chemins spécifiés.")

# Fonction pour charger les données
def load_data(file_path):
    data = pd.read_csv(file_path, sep=';')
    data['Total Revenue'] = data['Total Revenue'].fillna(0)
    data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
    data = data.sort_values(by='Date')
    df = data[['Date', 'Total Revenue']]
    df.set_index('Date', inplace=True)
    return df

# Préparation des données pour les modèles
def prepare_data(df, time_step):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df)

    X, y = [], []
    for i in range(len(scaled_data) - time_step):
        X.append(scaled_data[i:(i + time_step), 0])
        y.append(scaled_data[i + time_step, 0])

    X, y = np.array(X), np.array(y)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    return X, y, scaler

# Création du modèle LSTM
def create_lstm_model(time_step):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(time_step, 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Création du modèle XGBoost
def create_xgb_model():
    model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
    return model

# Fonction pour effectuer la prédiction
def predict(model, X, scaler):
    prediction = model.predict(X)
    prediction = scaler.inverse_transform(prediction.reshape(-1, 1))
    return prediction

@app.route('/')
def index():
    return render_template_string('''
    <h1>Prédiction des Ventes</h1>
    <form action="/predict" method="post" id="predictForm">
        <label for="model">Choisir le modèle:</label>
        <select name="model" id="model">
            <option value="lstm">LSTM</option>
            <option value="xgboost">XGBoost</option>
        </select><br><br>
        <label for="epochs">Nombre d'epochs (LSTM uniquement):</label>
        <input type="number" id="epochs" name="epochs" value="100"><br><br>
        <label for="time_step">Time step:</label>
        <input type="number" id="time_step" name="time_step" value="10"><br><br>
        <label for="train_split">Répartition entraînement/test (%):</label>
        <input type="number" id="train_split" name="train_split" value="70"><br><br>
        <input type="submit" value="Faire des Prédictions">
    </form>
    <div id="loading" style="display:none;">
        <img src="https://i.imgur.com/LLFzC6D.gif" alt="loading" width="50px">
        <p>Chargement en cours...</p>
    </div>
    <div id="result"></div>
    <script>
        document.getElementById("predictForm").onsubmit = function() {
            document.getElementById("loading").style.display = "block";
            document.getElementById("result").innerHTML = "";
        }
    </script>
    ''')

@app.route('/predict', methods=['POST'])
def predict_sales():
    model_choice = request.form['model']
    epochs = int(request.form['epochs'])
    time_step = int(request.form['time_step'])
    train_split = int(request.form['train_split']) / 100

    df = load_data(file_path)
    X, y, scaler = prepare_data(df, time_step)

    # Split des données en train et test
    train_size = int(len(X) * train_split)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    if model_choice == 'lstm':
        model = create_lstm_model(time_step)
        model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)
    elif model_choice == 'xgboost':
        model = create_xgb_model()
        model.fit(X_train.reshape(X_train.shape[0], X_train.shape[1]), y_train)

    test_predict = predict(model, X_test, scaler)

    plt.figure(figsize=(12, 6))
    plt.plot(df.index[time_step:], scaler.inverse_transform(y.reshape(-1, 1)), label='Données Originales')
    plt.plot(df.index[train_size+time_step:], test_predict, label='Prédictions ' + model_choice.upper())
    plt.xlabel('Date')
    plt.ylabel('Total des Revenus')
    plt.legend()

    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()
    plt.close()

    return render_template_string('''
    <h1>Prédiction des Ventes</h1>
    <img src="data:image/png;base64,{{ plot_url }}">
    <br>
    <a href="/">Retour</a>
    <script>
        document.getElementById("loading").style.display = "none";
    </script>
    ''', plot_url=plot_url)

if __name__ == '__main__':
    # Démarrer ngrok pour rendre le site accessible
    public_url = ngrok.connect(5000)
    print(f'Public URL: {public_url}')
    app.run(port=5000)



Public URL: NgrokTunnel: "https://3297-34-85-133-139.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 20:38:19] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 20:38:20] "GET /favicon.ico HTTP/1.1" 404 -


3/3 [==============================] - 1s 9ms/step


INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 20:38:56] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 20:39:08] "GET / HTTP/1.1" 200 -
